In [1]:
import pandas as pd
from PyOptimalInterpolation import get_data_path, get_parent_path
from PyOptimalInterpolation.local_experts import LocalExpertOI, get_results_from_h5file
from PyOptimalInterpolation.postprocessing import smooth_hyperparameters, SmoothingConfig
from PyOptimalInterpolation.config_dataclasses import (DataConfig, 
                                                       ModelConfig,
                                                       PredictionLocsConfig,
                                                       ExpertLocsConfig,
                                                       RunConfig,
                                                       LocalExpertConfig)


2023-06-01 18:50:14.134982: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 18:50:14.297315: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-01 18:50:14.811307: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:/usr/local/cuda-11.4/lib64
2023-06-01 18:50:14.811370: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; d

# Set up data/model/location configs

In [2]:
data_config = DataConfig(data_source=get_data_path("example", "ABC_binned_50km.h5"),
                         table="data",
                         obs_col="obs",
                         coords_col=["x", "y", "t"],
                         local_select=[{"col": "t", "comp": "<=", "val": 4},
                                       {"col": "t", "comp": "<=", "val": -4},
                                       {"col": ["x", "y"], "comp": "<", "val": 300_000}],
                         global_select=[{"col": "lat", "comp": ">=", "val": 60},
                                        {"loc_col": "t", "src_col": "date", "func": "lambda x,y: np.datetime64(pd.to_datetime(x+y, unit='D'))"}]
                        )


model_config = ModelConfig(oi_model="GPflowGPRModel",
                           init_params={"coords_scale": [50_000, 50_000, 1]},
                           constraints={"lengthscales": {"low": [1e-08, 1e-08, 1e-08],
                                                         "high": [600_000, 600_000, 9]}
                                    }
                           )


pred_locs_config = PredictionLocsConfig(method="from_dataframe",
                                       df_file=get_data_path("locations", "2d_xy_grid_5x5km.csv"),
                                       max_dist=200_000
                                       )


expert_locs_config = ExpertLocsConfig(source=get_data_path("locations", "example_expert_locations_arctic.csv"),
                                     add_data_to_col={"date": ["2020-03-04", "2020-03-05", "2020-03-06"]},
                                     col_funcs={"date": {"func": "lambda x: x.astype('datetime64[D]')",
                                                         "col_args": "date"
                                                      },
                                                "t": {"func": "lambda x: x.astype('datetime64[D]').astype(float)",
                                                      "col_args": "date"}
                                             },
                                     col_select=["x", "y", "t", "date", "lon", "lat"],
                                     row_select=[{"col": "date", "comp": "==", "val": "2020-03-05"},
                                                 {"col": "lat", "comp": ">=", "val": 60}],
                                     sort_by="date"
                                    )



In [3]:
run_config = RunConfig(store_path=get_parent_path("results", "example", "ABC_50km_test.h5"),
                       predict=False)

local_expert_config = LocalExpertConfig(data_config,
                                        model_config,
                                        expert_locs_config,
                                        pred_locs_config)


# Train model

Once you specify the config classes, training is easy.

In [4]:
# TODO: include train only option + fixed params option
locexp = LocalExpertOI(local_expert_config=local_expert_config)
locexp.run(**run_config.to_dict())

'data_select': 0.000 seconds
'load': 0.011 seconds
current config matched previous - idx: 1
---------
storing expert locations in 'expert_locs' table
closing
'data_select': 0.018 seconds
'load': 0.020 seconds
for table: expert_locs returning 0 / 363 entries
---------
dropping expert locations that already exists in 'run_details' table
closing
'data_select': 0.020 seconds
'load': 0.021 seconds
for table: run_details returning 0 / 363 entries
'run': 0.201 seconds


In [5]:
# See the tables in the results file
with pd.HDFStore(run_config.store_path, mode="r") as store:
    all_keys = store.keys()
    print(f"Columns = {all_keys}")

Columns = ['/expert_locs', '/kernel_variance', '/lengthscales', '/likelihood_variance', '/oi_config', '/run_details']


In [6]:
dfs, oi_configs = get_results_from_h5file(results_file=run_config.store_path,
                                          global_col_funcs=None,
                                          merge_on_expert_locations=False,
                                          select_tables=["lengthscales", "likelihood_variance", "kernel_variance"]
                                         )

reading in results
selecting only tables: ['lengthscales', 'likelihood_variance', 'kernel_variance']
'data_select': 0.000 seconds
'load': 0.005 seconds
expert_locations data will not be merged on results data


In [7]:
# Plot trained hyperparameters here


# Smooth hyperparameters

In [7]:
out_file = get_parent_path("results", "example", "ABC_50km_test_SMOOTHED.h5") # Path to store smoothed hyperparameters
smooth_configs = {"lengthscales": SmoothingConfig(l_x=200_000, l_y=200_000, max=12),
                  "likelihood_variance": SmoothingConfig(l_x=200_000, l_y=200_000),
                  "kernel_variance": SmoothingConfig(l_x=200_000, l_y=200_000, max=0.1)}

smooth_hyperparameters(result_file=run_config.store_path,
                       params_to_smooth=["lengthscales", "likelihood_variance", "kernel_variance"],
                       smooth_config_dict=smooth_configs,
                       output_file=out_file,
                       save_config_file=False)


reading in results
selecting only tables: ['lengthscales', 'likelihood_variance', 'kernel_variance', 'expert_locs', 'oi_config']
expert_locations data will not be merged on results data
adding smoothed table: lengthscales_SMOOTHED
adding smoothed table: likelihood_variance_SMOOTHED
adding smoothed table: kernel_variance_SMOOTHED
writing (smoothed) hyper parameters to:
/home/so/Documents/Projects/PyOptimalInterpolation/results/example/ABC_50km_test_SMOOTHED.h5
table_suffix:_SMOOTHED
writing: lengthscales_SMOOTHED to table
writing: likelihood_variance_SMOOTHED to table
writing: kernel_variance_SMOOTHED to table


In [8]:
# See the tables in the results file after smoothing
with pd.HDFStore(out_file, mode="r") as store:
    all_keys = store.keys()
    print(f"Columns = {all_keys}")

Columns = ['/kernel_variance_SMOOTHED', '/lengthscales_SMOOTHED', '/likelihood_variance_SMOOTHED']


In [9]:
dfs, oi_configs = get_results_from_h5file(results_file=out_file,
                                          global_col_funcs=None,
                                          merge_on_expert_locations=False,
                                          select_tables=["lengthscales", "likelihood_variance", "kernel_variance"],
                                          table_suffix="_SMOOTHED",
                                          add_suffix_to_table=True)

issuing getting oi_config_SMOOTHED
reading in results
selecting only tables: ['lengthscales_SMOOTHED', 'likelihood_variance_SMOOTHED', 'kernel_variance_SMOOTHED']
in get_results_from_h5file trying read expert_locations from file got Exception:
No objects to concatenate
expert_locations data will not be merged on results data


In [10]:
# Plot smoothed hyperparameters here

# Make predictions

To make predictions from the smoothed parameters, we re-run with a slightly different config

In [11]:
local_expert_config.model_config.load_params = {"file": out_file, "table_suffix": "_SMOOTHED"}
locexp = LocalExpertOI(local_expert_config=local_expert_config)

run_config.optimise = False
run_config.predict = True
run_config.table_suffix = "_SMOOTHED"

locexp.run(**run_config.to_dict())

'data_select': 0.000 seconds
'load': 0.007 seconds
---------
storing expert locations in 'expert_locs' table
exception occurred: 'No object named expert_locs_SMOOTHED in the file'
will now close object

---------
dropping expert locations that already exists in 'run_details' table
exception occurred: 'No object named run_details_SMOOTHED in the file'
will now close object

------------------------------
1 / 363
          x          y        t       date        lon        lat
0 -800000.0 -2000000.0  18326.0 2020-03-05 -21.801409  70.615022
'data_select': 0.027 seconds
'load': 0.028 seconds
'_update_global_data': 0.028 seconds
'local_data_select': 0.002 seconds
number obs: 0
------------------------------
2 / 363
             x          y        t       date         lon        lat
246 -1600000.0  1000000.0  18326.0 2020-03-05 -122.005383  73.040728
'_update_global_data': 0.001 seconds
'local_data_select': 0.002 seconds
number obs: 20
'_max_dist_bool': 0.014 seconds
'_from_dataframe': 0.4

2023-06-01 18:50:42.602348: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 18:50:45.497850: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 6261 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:17:00.0, compute capability: 7.5
2023-06-01 18:50:45.498613: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:1 with 4070 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:65:00.0, compute capability: 7.5
2023-06-01 18:50:45.507662: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6261 MB memory:  -> device: 0, name: N

'__init__': 3.148 seconds
'get_parameters': 0.005 seconds
'_read_params_from_file': 0.076 seconds
'set_parameters': 0.008 seconds
'load_params': 0.085 seconds
'set_lengthscales_constraints': 0.006 seconds
*** not optimising parameters


2023-06-01 18:50:46.372377: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x557091c98140


'get_parameters': 0.008 seconds
parameters:
lengthscales: array([3.74067836, 1.56982986, 0.99989855]) 
kernel_variance: 0.017064785656721305
likelihood_variance: 0.002319242371839619
'predict': 0.066 seconds
total run time : 4.76 seconds
------------------------------
3 / 363
             x          y        t       date         lon        lat
245 -1800000.0  1000000.0  18326.0 2020-03-05 -119.054604  71.477702
'_update_global_data': 0.001 seconds
'local_data_select': 0.002 seconds
number obs: 9
'_max_dist_bool': 0.015 seconds
'_from_dataframe': 0.016 seconds
found GPU
setting lengthscales to: [1. 1. 1.]


...

'__init__': 0.225 seconds
'_read_params_from_file': 0.084 seconds
'set_parameters': 0.007 seconds
'load_params': 0.093 seconds
'set_lengthscales_constraints': 0.005 seconds
*** not optimising parameters
'get_parameters': 0.004 seconds
parameters:
lengthscales: array([7.11625034, 5.55212456, 1.00030993]) 
kernel_variance: 0.004832699202286942
likelihood_variance: 0.0012853222729915425
'predict': 0.048 seconds
total run time : 0.70 seconds
------------------------------
16 / 363
            x         y        t       date         lon        lat
232 -600000.0  800000.0  18326.0 2020-03-05 -143.130102  81.037096
'_update_global_data': 0.001 seconds
'local_data_select': 0.002 seconds
number obs: 63
'_max_dist_bool': 0.014 seconds
'_from_dataframe': 0.017 seconds
found GPU
setting lengthscales to: [1. 1. 1.]


2023-06-01 18:50:55.498865: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 6261 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:17:00.0, compute capability: 7.5
2023-06-01 18:50:55.500031: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:1 with 4070 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:65:00.0, compute capability: 7.5


'__init__': 0.216 seconds
'_read_params_from_file': 0.075 seconds
'set_parameters': 0.006 seconds
'load_params': 0.082 seconds
'set_lengthscales_constraints': 0.006 seconds
*** not optimising parameters
'get_parameters': 0.004 seconds
parameters:
lengthscales: array([7.69557124, 4.72927882, 1.00009285]) 
kernel_variance: 0.007967085205425092
likelihood_variance: 0.0015709503373135235
'predict': 0.066 seconds
total run time : 0.71 seconds
------------------------------
17 / 363
            x         y        t       date    lon        lat
231 -800000.0  800000.0  18326.0 2020-03-05 -135.0  79.856495
'_update_global_data': 0.001 seconds
'local_data_select': 0.002 seconds
number obs: 73
'_max_dist_bool': 0.014 seconds
'_from_dataframe': 0.016 seconds
found GPU
setting lengthscales to: [1. 1. 1.]
'__init__': 0.178 seconds


2023-06-01 18:50:56.188295: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 6261 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:17:00.0, compute capability: 7.5
2023-06-01 18:50:56.188662: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:1 with 4070 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:65:00.0, compute capability: 7.5


'_read_params_from_file': 0.073 seconds
'set_parameters': 0.006 seconds
'load_params': 0.080 seconds
'set_lengthscales_constraints': 0.005 seconds
*** not optimising parameters
'get_parameters': 0.004 seconds
parameters:
lengthscales: array([6.64795434, 4.97055267, 0.99990784]) 
kernel_variance: 0.012256751904291666
likelihood_variance: 0.001982335595044621
'predict': 0.087 seconds
total run time : 0.69 seconds
------------------------------
18 / 363
             x         y        t       date         lon       lat
230 -1000000.0  800000.0  18326.0 2020-03-05 -128.659808  78.51376
'_update_global_data': 0.001 seconds
'local_data_select': 0.002 seconds
number obs: 66
'_max_dist_bool': 0.014 seconds
'_from_dataframe': 0.016 seconds
found GPU
setting lengthscales to: [1. 1. 1.]


In [ ]:
# Plot predictions